In [1]:
import pandas as pd
import numpy as np
import sys
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 1000)
import matplotlib.pyplot as plt
import os
import getpass
import seaborn as sns

In [2]:
if getpass.getuser() in ['ygnmax']:
    if sys.platform == 'linux':
        workdir = '/home/ygnmax/Dropbox/research_nyu/hedge_vol/'
    if sys.platform == 'win32':
        workdir = 'C:/Users/ygnmax/Dropbox/research_nyu/hedge_vol/'

In [3]:
##############################
# Get stock list / dictionary
##############################
df_bubble = pd.read_excel(workdir + "data/dot_com_firms.xlsx", engine = 'openpyxl').dropna()
df_bubble["SecurityID"] = df_bubble["SecurityID"].astype(int)
df_big = pd.read_excel(workdir + "data/big_firms.xlsx", engine = 'openpyxl').dropna()
df_big["SecurityID"] = df_big["SecurityID"].astype(int)
# read other companies
df_other = pd.read_excel(workdir + "data/other_firms.xlsx").dropna()
df_other["SecurityID"] = df_other["SecurityID"].astype(int)

df_stock_list = pd.concat([df_bubble[['Name','Ticker','SecurityID', 'Internet']], 
                           df_big[['Name','Ticker','SecurityID', 'Internet']], 
                          df_other[['Name','Ticker','SecurityID', 'Internet']]])
df_stock_list["Internet"] = df_stock_list["Internet"].astype(int)
df_stock_list = df_stock_list.dropna()
df_stock_list = df_stock_list.reset_index(drop = True)

ids = {}
dict_tech_label = {}
for i in list(df_stock_list.index):
    ids[df_stock_list.loc[i, 'SecurityID']] = df_stock_list.loc[i, 'Name']
    dict_tech_label[df_stock_list.loc[i, 'SecurityID']] = df_stock_list.loc[i, 'Internet']

In [4]:
MEDIUM_SIZE = 18
BIGGER_SIZE = 20
plt.rc('font', size=BIGGER_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=BIGGER_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=BIGGER_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=MEDIUM_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=MEDIUM_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=MEDIUM_SIZE)    # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title

# Plot Delta

In [9]:
adjusted_delta_path = 'adjusted_delta/'
input_path = workdir + 'data/processed/intermediate/WRDS_2021/' + 'delta_'
delta_var = 'Delta_c'
output_path = workdir + 'output/figures/'
if not os.path.exists(output_path):
    os.makedirs(output_path)
    
# test_date_begin_pre = pd.to_datetime("1999-09-09")
# turning_day = pd.to_datetime("2000-03-10")
# test_date_begin_post = pd.to_datetime("2001-06-28")   

test_date_begin_pre = pd.to_datetime("2020-01-01")
turning_day = pd.to_datetime("2021-01-28")
test_date_begin_post = pd.to_datetime("2021-06-28")  

j = 0
for i in [113993]: # ids:
    j = j + 1
    print('*********************************************')
    print('No.' + str(j) + ' processing '+ str(i))     
    if os.path.exists(input_path + str(i) + '.csv') == False:
        print('Stock '+ str(i) + ' is not available')
        continue
    else:
        temp = pd.read_csv(input_path + str(i) + ".csv", parse_dates = ['Date'])
        df_C = temp[(temp['CallPut'] == 'C') & (temp['M0'] < 1) & (temp['Delta_c'] < 1.01) & (temp['Delta_c'] > 0.01)]
        df_P = temp[(temp['CallPut'] == 'P') & (temp['M0'] > 1) & (temp['Delta_c'] > -1.01) & (temp['Delta_c'] < -0.01)]
        securityid = i
        tickername = df_stock_list.loc[df_stock_list['SecurityID'] == securityid, 'Ticker'].values[0].strip()
        fig, axs = plt.subplots(1, 2, figsize=(16,8), sharex=False, sharey=False)
        axs[0].scatter(df_C['Delta'], df_C['Delta_c'])
        axs[0].set_title("Call")
        axs[0].set_xlabel("Raw Delta", fontsize=20) 
        axs[0].set_ylabel("Adjusted Delta", fontsize=20) 
        axs[1].scatter(df_P['Delta'], df_P['Delta_c'])
        axs[1].set_title("Put")
        axs[1].set_xlabel("Raw Delta", fontsize=20)  
        fig.tight_layout()
        if not os.path.exists(output_path + str(securityid)):
            os.makedirs(output_path + str(securityid))
        plt.savefig(output_path + securityid + '/' + str(securityid) + '_' + tickername + '_delta.jpg')
        plt.close()

*********************************************
No.1 processing 113993


In [9]:
df_C[(df_C['CallPut'] == 'C') & (df_C['Delta_c'] > df_C['IV0_c'] ) & (df_C['IV0_c'] < 0.2)]

,Date,K,Expiration,CallPut,BestBid,BestOffer,LastTradeDate,Volume,IV0,Delta,Gamma,Vega,Theta,OptionID,V0,Maturity,tau,tau0,S0,AdjClosePrice,AdjClosePrice2,AdjustmentFactor,AdjustmentFactor2,S1,V1,IV1,Volume1,short_rate,r,M0,S0_n,V0_n,K_n,S1_n,V1_n,on_ret,SecurityID,StockTicker,impl_div30,impl_cdiv30,impl_div60,impl_cdiv60,impl_div90,impl_cdiv90,real_div,real_div_yield,impl_cdiv_median,IV0_c,V_self,V_up,V_down,Delta_c
316,2019-01-08,17.0,2021-01-15,C,2.25,2.78,2019-01-08,2.0,0.438683,0.479572,0.046088,7.854753,-1.244948,123878627,2.515,738,2.050000,1.466667,15.81,15.81,15.279742,2.0,2.877128,15.98,2.575,0.435328,2.0,0.024416,0.026934,0.930000,100.0,15.907653,107.526882,101.075269,16.287160,1.000068,113993,GME,-0.127728,-0.128383,0.022355,0.022376,-0.071150,-0.071695,0.0,0.0,-0.071695,0.107316,2.518958,2.670910,2.370116,0.951275
692,2019-01-16,17.0,2021-01-15,C,2.30,3.45,2019-01-16,5.0,0.498389,0.495133,0.040181,7.683317,-1.357697,123878627,2.875,730,2.027778,1.450000,15.68,15.68,15.154102,2.0,2.877128,15.77,2.700,0.469383,0.0,0.024397,0.026751,0.922353,100.0,18.335459,108.418367,100.573980,17.219388,1.000068,113993,GME,-0.529436,-0.541442,0.057585,0.057368,-0.064125,-0.064554,0.0,0.0,-0.064554,0.192590,2.869598,2.994925,2.745914,0.794041
3465,2019-04-09,10.0,2020-01-17,C,1.20,1.41,2019-04-09,21.0,0.497809,0.515661,0.110452,3.122315,-1.415709,117238899,1.305,283,0.786111,0.563889,9.82,9.82,9.820000,2.0,2.976974,9.97,1.365,0.493186,9.0,0.024451,0.025530,0.982000,100.0,13.289206,101.832994,101.527495,13.900204,1.000068,113993,GME,-0.258249,-0.261041,0.026803,0.026799,-0.151345,-0.154196,0.0,0.0,-0.154196,0.091819,1.308274,1.413228,1.204946,1.060500
5836,2019-07-12,5.0,2019-10-18,C,0.45,0.64,2019-07-12,20.0,0.655937,0.531819,0.274824,0.923018,-1.471421,127919235,0.545,98,0.272222,0.194444,4.91,4.91,4.910000,2.0,2.976974,4.75,0.475,0.684745,106.0,0.024143,0.022314,0.982000,100.0,11.099796,101.832994,96.741344,9.674134,1.000067,113993,GME,-0.423259,-0.430880,-0.374679,-0.386842,-0.599349,-0.649235,0.0,0.0,-0.430880,0.117733,0.557011,0.610337,0.504464,1.078138
5843,2019-07-12,5.0,2020-01-17,C,0.60,1.04,2019-07-12,25.0,0.774409,0.543642,0.181608,1.233785,-1.352371,117238910,0.820,189,0.525000,0.375000,4.91,4.91,4.910000,2.0,2.976974,4.75,0.730,0.780894,19.0,0.024143,0.021383,0.982000,100.0,16.700611,101.832994,96.741344,14.867617,1.000067,113993,GME,-0.423259,-0.430880,-0.374679,-0.386842,-0.599349,-0.649235,0.0,0.0,-0.430880,0.061278,1.212178,1.273741,1.150614,1.253841
5844,2019-07-12,6.0,2020-01-17,C,0.30,0.55,2019-07-12,21.0,0.691462,0.355559,0.179145,1.191887,-1.060703,128784014,0.425,189,0.525000,0.375000,4.91,4.91,4.910000,2.0,2.976974,4.75,0.445,0.768166,51.0,0.024143,0.021383,0.818333,100.0,8.655804,122.199593,96.741344,9.063136,1.000067,113993,GME,-0.423259,-0.430880,-0.374679,-0.386842,-0.599349,-0.649235,0.0,0.0,-0.430880,0.172024,0.424737,0.465020,0.386221,0.802434
5862,2019-07-15,5.0,2019-10-18,C,0.45,0.50,2019-07-15,106.0,0.684745,0.488788,0.277694,0.891527,-1.509742,127919235,0.475,95,0.263889,0.191667,4.75,4.75,4.750000,2.0,2.976974,4.63,0.455,0.733877,231.0,0.024072,0.022135,0.950000,100.0,10.000000,105.263158,97.473684,9.578947,1.000067,113993,GME,-1.358790,-1.442043,-0.468905,-0.488195,-0.565968,-0.610165,0.0,0.0,-0.610165,0.081828,0.609152,0.664845,0.553572,1.171297
5869,2019-07-15,5.0,2020-01-17,C,0.51,0.95,2019-07-15,19.0,0.780894,0.513486,0.187951,1.205758,-1.329279,117238910,0.730,186,0.516667,0.372222,4.75,4.75,4.750000,2.0,2.976974,4.63,0.585,0.711756,0.0,0.024072,0.021250,0.950000,100.0,15.368421,105.263158,97.473684,12.315789,1.000067,113993,GME,-1.358790,-1.442043,-0.468905,-0.488195,-0.565968,-0.610165,0.0,0.0,-0.610165,0.052327,1.564967,1.630071,1.499864,1.370604
5914,2019-07-17,5.0,2019-10-18,C,0.28,0.45,2019-07-17,673.0,0.760262,0.405277,0.263097,0.794589,-1.489985,127919235,0.365,93,0.258333,0.186111,4.38,4.38,4.380000,2.0,2.976974,4.20,0.305,0.783800,247.0,0.024025,0.022198,0.876000,100.0,8.333333,114